## Load data

In [1]:
from LoadSplitData import Process

Using TensorFlow backend.


In [2]:
data = Process()
data.split_data()

Images shape (24076, 80, 80, 3), label shape (24076,), ratio of authorized data 0.4503655092208008
X_train shape: (12639, 80, 80, 3)
12639 train samples
5418 valid samples
6019 test samples


## Train the model

In [3]:
from BuildModel import Model

In [4]:
model = Model()
model.build_model(data)
model.train(data)
model.save()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 80, 80, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 80, 80, 32)    9248        convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 40, 40, 32)    0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 40, 40, 32)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Evaluate the Model

In [5]:
model = Model()
model.load()
model.evaluate(data)

Model Loaded.
acc: 99.63%


## Predict new image

In [6]:
import cv2
import imutils
resized_height = 80

In [7]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\sample1.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[  9.65770505e-06   9.99990344e-01]]
1/1 [==============================] - 0s


1

In [8]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\sample2.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.01148414  0.98851585]]
1/1 [==============================] - 0s


1

In [9]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\sample3.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.26901844  0.73098159]]
1/1 [==============================] - 0s


1

In [10]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\sample4.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.02372973  0.97627026]]
1/1 [==============================] - 0s


1

In [11]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\chendaoming.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[  1.14444956e-04   9.99885559e-01]]
1/1 [==============================] - 0s


1

In [12]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\zhourunfa.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.29528266  0.70471728]]
1/1 [==============================] - 0s


1

In [13]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\chenbaoguo.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[  9.99957800e-01   4.22387529e-05]]
1/1 [==============================] - 0s


0

In [14]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\zhangguoli.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.99849248  0.00150752]]
1/1 [==============================] - 0s


0

In [15]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\lixuejian.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.99731737  0.0026826 ]]
1/1 [==============================] - 0s


0

In [16]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\wangziwen.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.9021771   0.09782287]]
1/1 [==============================] - 0s


0

In [17]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\tangguoqiang.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.09001063  0.90998942]]
1/1 [==============================] - 0s


1

In [18]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\zhangyimou.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.18087915  0.81912088]]
1/1 [==============================] - 0s


1